# Power BI Semantic Model Connection (Local)

Connect to Power BI Semantic Models from a **local Python environment** using the REST API.

## Prerequisites
1. **Azure CLI authentication**: Run `az login` before starting
2. **Power BI Premium, PPU, or Fabric capacity**: Required for executing DAX queries
3. **Workspace access**: You need at least read access to the workspace and semantic model

> **Note**: The `semantic-link` library's XMLA-based functions (`read_table`, `list_tables`) only work inside Fabric notebooks. This notebook uses the REST API which works locally.

In [1]:
import pandas as pd
import requests
import json
from azure.identity import DefaultAzureCredential, DeviceCodeCredential
from azure.core.exceptions import ClientAuthenticationError

In [2]:
# Authenticate to Power BI
try:
    credential = DefaultAzureCredential()
    token = credential.get_token("https://analysis.windows.net/powerbi/api/.default")
    print("✓ Authenticated using DefaultAzureCredential")
except ClientAuthenticationError:
    print("DefaultAzureCredential failed. Using DeviceCodeCredential...")
    credential = DeviceCodeCredential()
    token = credential.get_token("https://analysis.windows.net/powerbi/api/.default")
    print("✓ Authenticated using DeviceCodeCredential")

✓ Authenticated using DefaultAzureCredential


## Power BI REST API Helper Class

A reusable class for interacting with Power BI semantic models from local Python.

In [10]:
from powerbi_client import PowerBIClient

# Initialize the client
pbi = PowerBIClient(credential)
print("✓ PowerBIClient initialized")

✓ PowerBIClient initialized


## 1. List Workspaces and Datasets

In [4]:
# List workspaces
df_workspaces = pbi.list_workspaces()
print(f"Found {len(df_workspaces)} workspaces")
display(df_workspaces[["name", "id", "isOnDedicatedCapacity"]].head(10))

Found 1 workspaces


,name,id,isOnDedicatedCapacity
0,perry-pbi-demo-workspace,52cb91e1-0aef-4d50-98b7-b65b748ed997,True


In [5]:
# Configuration - set your workspace and dataset names
WORKSPACE_NAME = "perry-pbi-demo-workspace"
DATASET_NAME = "Customer Profitability Sample"

# Check if workspace is on Premium capacity
if pbi.is_premium(WORKSPACE_NAME):
    print(f"✓ Workspace '{WORKSPACE_NAME}' is on Premium/Fabric capacity")
else:
    print(f"✗ Workspace '{WORKSPACE_NAME}' is NOT on Premium capacity - DAX queries will fail!")

# List datasets in workspace
df_datasets = pbi.list_datasets(WORKSPACE_NAME)
print(f"\nDatasets in '{WORKSPACE_NAME}':")
display(df_datasets[["name", "id", "configuredBy"]])

✓ Workspace 'perry-pbi-demo-workspace' is on Premium/Fabric capacity

Datasets in 'perry-pbi-demo-workspace':


,name,id,configuredBy
0,Customer Profitability Sample,a99cf490-f23d-4dea-8dc2-399c6c8ffc40,blaineperry@microsoft.com


## Describe Dataset - Get Full Schema

In [6]:
# Test describe_dataset method
result = pbi.describe_dataset(WORKSPACE_NAME, DATASET_NAME)
print(f"Found {len(result['tables'])} tables\n")
print("=" * 60)
for table in result['tables']:
    print(f"\n📊 {table['name']} ({len(table['columns'])} columns)")
    for col in table['columns']:
        card = f"({col['cardinality']} unique)" if col.get('cardinality') else ""
        dtype = f"[{col.get('dataType', '?')}]"
        print(f"   └─ {col['name']} {dtype} {card}")

print("\n" + "=" * 60)
print("\n🔗 Inferred Relationships:")
for rel in result['relationships']:
    print(f"   {rel['keyColumn']}: {' <-> '.join(rel['tables'])}")

print("\n" + "=" * 60)
print("\n📝 LLM Context (first 2000 chars):")
print(result['llm_context'][:2000])

Found 9 tables


📊 Fact (13 columns)
   └─ Customer Key [Number] (90 unique)
   └─ Product Key [Text] (7 unique)
   └─ BU Key [Number] (26 unique)
   └─ Scenario Key [Number] (2 unique)
   └─ Revenue [Number] (9805 unique)
   └─ Material Costs [Number] (5 unique)
   └─ Labor Costs Variable [Number] (2250 unique)
   └─ Taxes [Unknown] (5 unique)
   └─ Rev for Exp Travel [Number] (4240 unique)
   └─ Travel Expenses [Number] (4286 unique)
   └─ Cost Third Party [Number] (3458 unique)
   └─ Subscription Revenue [Number] (9 unique)
   └─ YearPeriod [Text] (16 unique)

📊 BU (4 columns)
   └─ BU Key [Number] (164 unique)
   └─ BU [Text] (37 unique)
   └─ Division [Text] (3 unique)
   └─ Executive_id [Number] (8 unique)

📊 Calendar (7 columns)
   └─ YearPeriod [Text] (84 unique)
   └─ Year [Number] (7 unique)
   └─ Period [Text] (12 unique)
   └─ Date [DateTime] (84 unique)
   └─ Month [Text] (12 unique)
   └─ QtrID [Text] (28 unique)
   └─ Qtr [Text] (4 unique)

📊 Scenario (2 columns)
   └─ S

## 2. Read Data from a Table

In [7]:
# Read the first 10 rows from the 'Customer' table
df_customers = pbi.read_table(WORKSPACE_NAME, DATASET_NAME, "Customer", top_n=10)
print(f"Retrieved {len(df_customers)} rows from 'Customer' table")
display(df_customers)

Retrieved 10 rows from 'Customer' table


,Customer[Customer],Customer[Name],Customer[City],Customer[Postal Code],Customer[State],Customer[Industry ID],Customer[Country/Region]
0,1023.0,Spade and Archer,Irving,75038.0,TX,31.0,US
1,10000.0,Globo-Chem,Chicago,60601.0,IL,30.0,US
2,10001.0,SNC Directly to America,Westchester,60154.0,IL,30.0,US
3,10002.0,GHG,Plano,75024.0,TX,13.0,US
4,10003.0,ABC Helicopter,Fort Worth,76177.0,TX,34.0,US
5,10004.0,Mr. Sparkle,Toronto,NaN,ON,30.0,CA
6,10005.0,GAM Neuro,South Bend,46617.0,IN,34.0,US
7,10006.0,Sourced Out,Chestbrook,19087.0,PA,26.0,US
8,10007.0,Processes Inc,Foster City,94404.0,CA,30.0,US
9,10008.0,Cadams USA,EI Paso,79998.0,TX,7.0,US


## 3. Execute Custom DAX Query

In [8]:
# Execute a custom DAX query
dax_query = """
EVALUATE
SUMMARIZECOLUMNS(
    'Customer'[State],
    "Customer Count", COUNTROWS('Customer')
)
ORDER BY [Customer Count] DESC
"""

df_result = pbi.execute_dax(WORKSPACE_NAME, DATASET_NAME, dax_query)
print(f"Query returned {len(df_result)} rows")
display(df_result)

Query returned 37 rows


,Customer[State],[Customer Count]
0,TX,61
1,CA,32
2,VA,26
3,PA,24
4,OH,17
5,IL,16
6,FL,14
7,NY,14
8,NJ,11
9,NC,10


## 4. Evaluate a Measure

In [9]:
# Evaluate a measure (replace with an actual measure from your model)
# Example: Evaluate [Total Revenue] grouped by State
try:
    df_measure = pbi.evaluate_measure(
        WORKSPACE_NAME, 
        DATASET_NAME, 
        "[Total Revenue]",  # Replace with your measure name
        group_by=["'Customer'[State]"]
    )
    display(df_measure.head(10))
except Exception as e:
    print(f"Note: {e}")
    print("Tip: Replace '[Total Revenue]' with an actual measure from your semantic model")

,Customer[State],[Result]
0,NaN,1.010850e+06
1,TX,1.842507e+07
2,IL,8.339600e+07
3,ON,8.163074e+05
4,IN,4.328036e+05
5,PA,1.815052e+07
6,CA,1.714299e+06
7,VA,4.755316e+07
8,MO,2.667600e+05
9,MD,2.886703e+06
